In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import randint, uniform
import time
import warnings
warnings.filterwarnings('ignore')

# 데이터 로드
with open('df_selected_05.pkl', 'rb') as f:
    df = pickle.load(f)

# X, y 분리
y = df['SalePrice']
X = df.drop('SalePrice', axis=1)

print(f"데이터 형태: X {X.shape}, y {y.shape}")

# 교차 검증 설정
kfold = KFold(n_splits=5, shuffle=True, random_state=42)


데이터 형태: X (1460, 15), y (1460,)


In [2]:
# Random Search를 위한 파라미터 분포 정의
param_distributions = {
    'n_estimators': randint(100, 1000),           # 100~999 사이 정수
    'max_depth': [None] + list(range(5, 51, 5)),  # None 또는 5~50 (5 간격)
    'min_samples_split': randint(2, 21),          # 2~20 사이 정수
    'min_samples_leaf': randint(1, 11),           # 1~10 사이 정수
    'max_features': ['sqrt', 'log2'] + [0.3, 0.5, 0.7, 0.9, 1.0],  # 다양한 특성 비율
    'max_samples': uniform(0.6, 0.4),             # 0.6~1.0 사이 실수
    'bootstrap': [True, False]                     # 부트스트랩 사용 여부
}

print("=== 파라미터 분포 ===")
for param, distribution in param_distributions.items():
    print(f"{param}: {distribution}")
    
print(f"\n총 가능한 조합: 매우 많음 (연속 분포 포함)")
print(f"Random Search로 100회 시도 예정")


=== 파라미터 분포 ===
n_estimators: <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7787bcd4a9c0>
max_depth: [None, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
min_samples_split: <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7787bc9619a0>
min_samples_leaf: <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7787bb1cfa40>
max_features: ['sqrt', 'log2', 0.3, 0.5, 0.7, 0.9, 1.0]
max_samples: <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7787bb1cf050>
bootstrap: [True, False]

총 가능한 조합: 매우 많음 (연속 분포 포함)
Random Search로 100회 시도 예정


In [3]:
# 기준선 모델 성능
base_model = RandomForestRegressor(random_state=42)
base_scores = cross_val_score(base_model, X, y, cv=kfold, scoring='r2')
baseline_r2 = base_scores.mean()

print(f"기준선 R² 점수: {baseline_r2:.4f}")

# Random Search 설정
rf = RandomForestRegressor(random_state=42, n_jobs=-1)
random_search = RandomizedSearchCV(
    rf, 
    param_distributions,
    n_iter=100,          # 100회 시도
    cv=kfold,
    scoring='r2',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

print("\n=== Random Search 시작 ===")
start_time = time.time()
random_search.fit(X, y)
end_time = time.time()

print(f"\nRandom Search 완료!")
print(f"소요 시간: {end_time - start_time:.2f}초")
print(f"최적 R² 점수: {random_search.best_score_:.4f}")
print(f"성능 향상: {random_search.best_score_ - baseline_r2:.4f}")
print(f"향상률: {((random_search.best_score_ - baseline_r2) / baseline_r2 * 100):.2f}%")


기준선 R² 점수: 0.8335

=== Random Search 시작 ===
Fitting 5 folds for each of 100 candidates, totalling 500 fits

Random Search 완료!
소요 시간: 77.63초
최적 R² 점수: 0.8426
성능 향상: 0.0091
향상률: 1.09%


In [4]:
print("=== 최적 하이퍼파라미터 ===")
best_params = random_search.best_params_
for param, value in best_params.items():
    print(f"{param}: {value}")

# 상위 10개 결과 분석
results_df = pd.DataFrame(random_search.cv_results_)
top_10 = results_df.nlargest(10, 'mean_test_score')[
    ['mean_test_score', 'std_test_score'] + [f'param_{p}' for p in param_distributions.keys()]
]

print("\n=== 상위 10개 결과 ===")
print(top_10.to_string(index=False))

# 최종 모델 검증
final_model = RandomForestRegressor(**best_params, random_state=42)
final_scores = cross_val_score(final_model, X, y, cv=kfold, scoring='r2')

print(f"\n=== 최종 검증 결과 ===")
print(f"R² 평균: {final_scores.mean():.4f} (±{final_scores.std():.4f})")
print(f"개별 폴드 점수: {final_scores}")


=== 최적 하이퍼파라미터 ===
bootstrap: True
max_depth: None
max_features: 0.7
max_samples: 0.6920741072966221
min_samples_leaf: 3
min_samples_split: 5
n_estimators: 646

=== 상위 10개 결과 ===
 mean_test_score  std_test_score  param_n_estimators param_max_depth  param_min_samples_split  param_min_samples_leaf param_max_features  param_max_samples  param_bootstrap
        0.842583        0.039551                 646            None                        5                       3                0.7           0.692074             True
        0.842567        0.038024                 739              10                       10                       4                0.5           0.858069             True
        0.842408        0.035591                 666              50                       13                       3                0.3           0.759944             True
        0.842321        0.038564                 769              35                       11                       2            

In [5]:
# 모델 저장
final_model.fit(X, y)
with open('quick_tuned_rf_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)

# 특성 중요도
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n=== 특성 중요도 (Top 5) ===")
print(feature_importance.head().to_string(index=False))

print("\n빠른 튜닝 모델이 'quick_tuned_rf_model.pkl'로 저장되었습니다.")



=== 특성 중요도 (Top 5) ===
       feature  importance
Quality_x_Area    0.470718
       TotalSF    0.171059
   OverallQual    0.155984
  EffectiveAge    0.037283
   TotalBsmtSF    0.033931

빠른 튜닝 모델이 'quick_tuned_rf_model.pkl'로 저장되었습니다.
